In [1]:
from test_bench import get_checkpoint_name, load_particle_datasets, subsample_dataset
from test_bench.model import MonteCarloDropoutModel


1. Select sample data
2. Create a model
3. Test the model

In [21]:
# Parameters
PARTICLE = 'pion'
CHECKPOINT_DP = 0.01
DROPOUT_TYPE = 'bernoulli_structured'
CHECKPOINT_BASE = 'checkpoints/'
DATA_DIR = 'rich/'
SUB_SAMPLE_PERCENT = 0.1

# MCD parameters
MCD_ENSEMBLE_SIZE = 300

#FD parameters


# Load data and Sample selection

In [3]:
dataset = load_particle_datasets(PARTICLE, DATA_DIR)

Reading and concatenating datasets:
	rich\pion2_+_down_2016_.csv
	rich\pion2_+_up_2016_.csv
	rich\pion2_-_down_2016_.csv
	rich\pion2_-_up_2016_.csv
	rich\pion_+_down_2016_.csv
	rich\pion_+_up_2016_.csv
	rich\pion_-_down_2016_.csv
	rich\pion_-_up_2016_.csv
splitting to train/val/test
fitting the scaler
scaler train sample size: 2000000
scaler n_quantiles: 100000, time = 2.2017691135406494
scaling train set
scaling test set
converting dtype to <class 'numpy.float32'>
feats_train shape	(948325, 3)
targets_train shape	(948325, 5)
feats_val shape  	(527302, 3)
targets_val shape	(527302, 5)



In [4]:
# Draw a sample of the datasets
x_sample, y_sample = subsample_dataset(dataset['feats_val'], dataset['targets_val'], SUB_SAMPLE_PERCENT)
x_sample.shape, y_sample.shape

(TensorShape([52730, 3]), TensorShape([52730, 5]))

# Model creation

In [5]:
model = MonteCarloDropoutModel(
    PARTICLE,
    dropout_rate=CHECKPOINT_DP,
    checkpoint_dir=CHECKPOINT_BASE + get_checkpoint_name(PARTICLE, CHECKPOINT_DP, DROPOUT_TYPE),
    debug=True
)
generator = model.get_generator()

Generating model for pion with a dropout rate of 0.01
Layer 0
Layer 1
Layer 2
Layer 3
Layer 4

Generator:

Model: "virtual_ensemble_model"
________________________________________________________________________________________________
 Layer (type)                              Output Shape                          Param #        
 Inputs (InputLayer)                       [(None, 3)]                           0              
                                                                                                
 NoiseInjection (NoiseInjection)           (None, 67)                            0              
                                                                                                
 Layer_0/Dense (Dense)                     (None, 128)                           8704           
                                                                                                
 Layer_0/LeakyReLU (LeakyReLU)             (None, 128)                           0   

## Single model prediction

In [6]:
generator.single_model_inference_mode()
t_generated = generator.predict(x_sample)
t_generated

1648/1648 [==============================] - 5s 2ms/step


array([[-1.6650505 , -0.49477583, -2.0765963 , -0.3902473 , -0.42617923],
       [ 1.9638599 , -1.3240287 ,  1.6830475 , -1.2790949 , -1.3487611 ],
       [ 1.1652783 ,  0.29276836,  0.6128771 ,  0.5450657 ,  0.46852428],
       ...,
       [-0.661861  , -0.8433812 , -0.77467686, -0.7675832 , -0.80289394],
       [ 0.99369395,  1.0196021 ,  0.3390171 ,  0.8842587 , -0.13664594],
       [-0.7586589 , -1.2008123 , -0.6705921 , -0.85414964, -0.9172192 ]],
      dtype=float32)

## MCD

In [24]:
from mcd.MCDEvaluator import evaluate_model

mcd_uncertainty, _ =  evaluate_model(model, x_sample, MCD_ENSEMBLE_SIZE)
mcd_uncertainty


Generating ensemble(300) predictions


100%|██████████| 300/300 [00:48<00:00,  6.14it/s]


<tf.Tensor: shape=(52730, 5), dtype=float32, numpy=
array([[0.2775522 , 0.386676  , 0.46374527, 0.39949757, 0.41708624],
       [0.34652832, 0.31579727, 0.32353157, 0.5058358 , 0.8578255 ],
       [0.31684008, 0.4330985 , 0.29499567, 0.48364782, 0.6793873 ],
       ...,
       [0.36786205, 0.4133227 , 0.58486474, 0.4431895 , 0.51973313],
       [0.0923771 , 0.33238778, 0.07875585, 0.4219105 , 0.5567071 ],
       [0.434161  , 0.54707956, 0.64875007, 0.5411884 , 0.5764225 ]],
      dtype=float32)>

In [25]:
mcd_uncertainty.shape

TensorShape([52730, 5])

## FD